In [423]:
import glob
import os
import json
import pickle
import yaml
import math

import numpy as np
import pandas as pd
pd.options.mode.chained_assignment = None
import pyarrow.parquet as pq
from sklearn.metrics import auc, roc_curve
from scipy.special import softmax

import hist as hist2
import matplotlib.pyplot as plt
import mplhep as hep

plt.style.use(hep.style.CMS)

import sys
sys.path
sys.path.append("../python/")

import utils

plt.rcParams.update({"font.size": 20})

import json
import pickle as pkl
import warnings
from typing import List

import numpy as np
import scipy
from hist import Hist

def get_finetuned_score(data, model_path):
    import onnx
    import onnxruntime as ort

    input_dict = {
        "highlevel": data.loc[:, "fj_ParT_hidNeuron000":"fj_ParT_hidNeuron127"].values.astype("float32"),
    }

    onnx_model = onnx.load(model_path)
    onnx.checker.check_model(onnx_model)

    ort_sess = ort.InferenceSession(
        model_path,
        providers=["AzureExecutionProvider"],
    )
    outputs = ort_sess.run(None, input_dict)

    return scipy.special.softmax(outputs[0], axis=1)[:, 0]

In [424]:
%load_ext autoreload
%autoreload 2
%reload_ext autoreload

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [425]:
! ls /Users/fmokhtar/Downloads/outfiles_again/outfiles/

ls: /Users/fmokhtar/Downloads/outfiles_again/outfiles/: No such file or directory


In [464]:
# parquets = glob.glob("../eos/May10_LP_2018/GluGluHToWW_Pt-200ToInf_M-125/outfiles/*_mu.parquet")
# parquets = glob.glob("/Users/fmokhtar/Downloads/outfiles/0-20_*.parquet")

# parquets = glob.glob("../eos/May23_LP_2017/TTToSemiLeptonic/outfiles/*.parquet")

parquets = glob.glob("/Users/fmokhtar/Downloads/outfiles/0-20*.parquet")
# parquets = glob.glob("/Users/fmokhtar/Downloads/outfiles_quarkisnolepnonu/0-5*.parquet")

df = pd.read_parquet(parquets)
df["THWW"] = get_finetuned_score(df, "../../weaver-core-dev/experiments_finetuning/v35_30/model.onnx")
score = df["THWW"].values

In [447]:
for key in df:
    print(key)

lep_pt
lep_eta
lep_isolation
lep_misolation
lep_fj_dr
lep_met_mt
met_fj_dphi
met_pt
deta
mjj
ht
n_bjets_L
n_bjets_M
n_bjets_T
fj_lsf3
NumFatjets
NumOtherJets
FirstFatjet_pt
FirstFatjet_eta
FirstFatjet_phi
FirstFatjet_msd
SecondFatjet_pt
SecondFatjet_eta
SecondFatjet_phi
SecondFatjet_msd
n_loose_electrons
n_tight_electrons
n_loose_muons
n_tight_muons
rec_higgs_m
rec_higgs_pt
rec_W_qq_m
rec_W_qq_pt
rec_W_lnu_m
rec_W_lnu_pt
fj_isTop
fj_isTop_matched
fj_Top_numMatched
fj_Top_nquarksnob
fj_Top_nbquarks
fj_Top_ncquarks
fj_Top_nleptons
fj_Top_nele
fj_Top_nmu
fj_Top_ntau
fj_Top_taudecay
fj_isTop_W_lep_b
fj_isTop_W_lep
fj_isTop_W_ele_b
fj_isTop_W_ele
fj_isTop_W_mu_b
fj_isTop_W_mu
fj_isTop_W_tau_b
fj_isTop_W_tau
fj_genjetmass
fj_genjetpt
weight_ewkcorr
weight_qcdcorr
weight_altqcdcorr
weight_scale0
weight_scale1
weight_scale3
weight_scale5
weight_scale7
weight_scale8
weight_scale4
weight_ele
weight_btag
weight_mu
LP_pfcand0_px
LP_pfcand0_py
LP_pfcand0_pz
LP_pfcand0_energy
LP_pfcand1_px
LP_pfcand

In [448]:
print("          nquarks     counts")
for thresh in [0.0, 0.2,0.4,0.6,0.8]:
    print(f"THWW>{thresh}: ", np.unique(df["num_quarks"][score>thresh], return_counts=True)[0], np.unique(df["num_quarks"][score>thresh], return_counts=True)[1])

          nquarks     counts


KeyError: 'num_quarks'

In [437]:
print("        nquarks (matched)   counts")
for thresh in [0.0, 0.2,0.4,0.6,0.8]:
    print(f"THWW>{thresh}: ", np.unique(df["num_m_quarks"][score>thresh], return_counts=True)[0], np.unique(df["num_m_quarks"][score>thresh], return_counts=True)[1])

        nquarks (matched)   counts
THWW>0.0:  [0 1 2] [147 539 878]
THWW>0.2:  [0 1 2] [ 27 267 812]
THWW>0.4:  [0 1 2] [  7 141 734]
THWW>0.6:  [0 1 2] [  3  71 622]
THWW>0.8:  [0 1 2] [  2  23 457]


In [438]:
print("        nquarks (matched)   counts")
for thresh in [0.0, 0.2,0.4,0.6,0.8]:
    perc = (100*np.unique(df["num_m_quarks"][score>thresh], return_counts=True)[1]/np.unique(df["num_m_quarks"][score>thresh], return_counts=True)[1].sum()).round()
    print(f"THWW>{thresh}: ", np.unique(df["num_m_quarks"][score>thresh], return_counts=True)[0], f"{perc}%")

        nquarks (matched)   counts
THWW>0.0:  [0 1 2] [ 9. 34. 56.]%
THWW>0.2:  [0 1 2] [ 2. 24. 73.]%
THWW>0.4:  [0 1 2] [ 1. 16. 83.]%
THWW>0.6:  [0 1 2] [ 0. 10. 89.]%
THWW>0.8:  [0 1 2] [ 0.  5. 95.]%


In [465]:
print("        nquarks (matched)   counts")
for thresh in [0.0, 0.2,0.4,0.6,0.8]:
    perc = (100*np.unique(df["fj_Top_nquarksnob"][score>thresh], return_counts=True)[1]/np.unique(df["fj_Top_nquarksnob"][score>thresh], return_counts=True)[1].sum()).round()
    print(f"THWW>{thresh}: ", np.unique(df["fj_Top_nquarksnob"][score>thresh], return_counts=True)[0], f"{perc}%")

        nquarks (matched)   counts
THWW>0.0:  [0 1 2] [89.  9.  2.]%
THWW>0.2:  [0 1 2] [61. 28. 11.]%
THWW>0.4:  [0 1 2] [52. 33. 16.]%
THWW>0.6:  [0 1 2] [41. 36. 23.]%
THWW>0.8:  [0 1 2] [31. 33. 36.]%


In [466]:
print("        nquarks (matched)   counts")
for thresh in [0.0, 0.2,0.4,0.6,0.8]:
    perc = (np.unique(df["fj_Top_nquarksnob"][score>thresh], return_counts=True)[1]).round()
    print(f"THWW>{thresh}: ", np.unique(df["fj_Top_nquarksnob"][score>thresh], return_counts=True)[0], f"{perc}")

        nquarks (matched)   counts
THWW>0.0:  [0 1 2] [5264  516  105]
THWW>0.2:  [0 1 2] [288 132  51]
THWW>0.4:  [0 1 2] [120  76  37]
THWW>0.6:  [0 1 2] [46 40 25]
THWW>0.8:  [0 1 2] [14 15 16]


## Retrieve the pfcands, genquark, and ak8 jet info

In [243]:
df

,lep_pt,lep_eta,lep_isolation,lep_misolation,lep_fj_dr,lep_met_mt,met_fj_dphi,met_pt,deta,mjj,...,fj_ParT_hidNeuron119,fj_ParT_hidNeuron120,fj_ParT_hidNeuron121,fj_ParT_hidNeuron122,fj_ParT_hidNeuron123,fj_ParT_hidNeuron124,fj_ParT_hidNeuron125,fj_ParT_hidNeuron126,fj_ParT_hidNeuron127,THWW
0,284.333710,0.020432,0.005537,0.000000,0.152496,112.509384,-0.684649,91.988869,0.536987,143.575012,...,-1.942991,0.930049,2.109063,1.496683,0.821488,-1.400893,-1.783140,2.865503,1.595134,0.235581
1,92.167236,-1.376465,0.079222,0.000000,0.402191,14.079594,0.189468,57.067398,0.513916,89.560562,...,-1.712093,1.025458,1.751106,1.389767,0.704361,-0.761926,-1.034831,3.399926,0.832115,0.014007
2,104.614265,1.902832,0.010866,0.000000,0.372378,116.946724,-1.518633,111.096214,0.851227,344.202606,...,-1.617962,0.700203,1.788390,0.882051,1.192969,-1.097186,-1.401819,3.712216,0.438131,0.002589
3,197.575607,-0.688354,0.013517,0.000000,0.319718,53.165493,0.606267,99.920547,3.387207,723.012390,...,-1.853088,1.052001,1.821456,1.407199,1.057116,-1.113967,-1.399664,3.930620,2.084403,0.001013
4,127.048485,-0.242096,0.563336,0.000000,0.136973,30.868221,0.240115,105.469971,0.732208,129.877121,...,-1.899507,0.864805,1.250199,1.407364,1.061866,-1.013456,-1.438947,3.206911,1.409576,0.008969
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
731291,189.033066,1.078613,0.016907,0.000000,0.080699,93.542641,-0.584959,114.154182,0.100342,72.288918,...,-1.481092,0.517991,1.193817,-0.247705,0.709992,-0.421738,-1.432085,0.703629,1.812942,0.088464
731292,116.969818,1.114502,0.000000,0.000000,0.204142,50.143772,0.489489,109.860146,0.002527,607.505371,...,-1.102406,0.575852,0.760395,0.059958,0.416334,-0.781419,-1.565744,2.128098,2.150358,0.025795
731293,124.680794,0.636108,0.614042,0.004771,0.172472,22.348181,-0.131554,88.547333,0.469727,145.417938,...,-0.840941,0.725677,0.676632,-0.167950,-0.008208,-0.724948,-1.403607,1.875653,2.087651,0.004378
731294,48.106216,0.759399,0.092118,0.010428,0.539897,72.334244,-0.674764,91.680656,0.594482,128.611389,...,-0.255430,0.058063,-0.078266,0.858702,-0.710914,-0.421021,-0.945352,1.510525,0.962455,0.040073


In [467]:
df = df[df["num_m_quarks"]==1]

KeyError: 'num_m_quarks'

In [485]:
df = df[df["fj_Top_nquarksnob"]==1]

KeyError: 'fj_Top_nquarks'

In [469]:
# (1) PFcands

pxs = df.loc[:,df.columns.str.contains("LP_pfcand") & df.columns.str.contains("px")].values
pys = df.loc[:,df.columns.str.contains("LP_pfcand") & df.columns.str.contains("py")].values
pzs = df.loc[:,df.columns.str.contains("LP_pfcand") & df.columns.str.contains("pz")].values
Es = df.loc[:,df.columns.str.contains("LP_pfcand") & df.columns.str.contains("energy")].values

pf_cands = np.stack([pxs,pys,pzs,Es], axis=-1)

print("pf_cands", pf_cands.shape)

pf_cands (516, 150, 4)


In [470]:
# (2) ak8 jet, and the THWW score

ak8_jets = np.stack([df["LP_fj_pt"].values, df["LP_fj_eta"].values, df["LP_fj_phi"].values, df["LP_fj_energy"].values], axis=-1)
print("ak8_jets", ak8_jets.shape)

score = df["THWW"].values
print("THWW", df["THWW"].shape)

ak8_jets (516, 4)
THWW (516,)


In [478]:
# (3) genQuarks

etas = df.loc[:,df.columns.str.contains("LP_quark") & df.columns.str.contains("eta")].values
phis = df.loc[:,df.columns.str.contains("LP_quark") & df.columns.str.contains("phi")].values

gen_parts_eta_phi = np.stack([etas,phis], axis=-1)
print("gen_parts_eta_phi (# events, # quarks, eta&phi)", gen_parts_eta_phi.shape)

gen_parts_eta_phi (# events, # quarks, eta&phi) (516, 2, 2)


In [483]:
gen_parts_eta_phi = gen_parts_eta_phi[:,1,:].reshape(-1,1,2)
gen_parts_eta_phi.shape

IndexError: index 1 is out of bounds for axis 1 with size 1

## Initiate a LundReweigher

In [484]:
import sys, os
sys.path.insert(0, '')
sys.path.append("LundReweighting")
from LundReweighting.utils.Utils import *

""" An example how to use the Lund Plane reweighting  code. 
Computes reweighting factors for an example signal. 
Uses them to compute the efficiency and uncertainty of a given substructure cut"""


######################## Setup 

#Input file 
fname = "LundReweighting/data/example_signal.h5"
f_ratio_name = 'LundReweighting/data/ratio_2017.root'

f_sig = h5py.File(fname, "r")
f_ratio = ROOT.TFile.Open(f_ratio_name)

#Class to help read input dataset 
d = Dataset(f_sig, dtype = 1)
d.compute_obs()

#The cut we will compute a SF for 'THWW>0.92

#Main class for reweighting utilities
LP_rw = LundReweighter(f_ratio = f_ratio)

#use small number of events for the example, but generally advisable to use larger sample

max_evts = -1

######## HWW
pf_cands = pf_cands[:max_evts]
gen_parts_eta_phi = gen_parts_eta_phi[:max_evts]
ak8_jets = ak8_jets[:max_evts]

################### Compute reweighting factors

#PF candidates in the AK8 jet for each event (list of (px,py,pz,E))
print("Running on %i events" % len(pf_cands))

#Nominal event weights of the MC, (assume every event is weight '1' for this example)
nom_weights = np.ones(len(pf_cands))

#Compute reweighting factors and all systematic variations
LP_weights = LP_rw.get_all_weights(pf_cands, gen_parts_eta_phi, ak8_jets, gen_parts_pdg_ids = None, distortion_sys=True, pf_cands_PtEtaPhiE_format=False)

#multiply Lund plane weights with nominal event weights
for key in LP_weights.keys():
    if('nom' in key or 'up' in key or 'down' in key):
        if(isinstance(LP_weights[key], np.ndarray)) : LP_weights[key] *= nom_weights


#Fraction of prongs that are not well matched to subjets (want this to be low)
print("Bad match frac %.2f" % np.mean(LP_weights['bad_match']))
#Fraction of prongs that are still not well matched after reclustering with varied number of prongs
print("Reclustered bad match frac %.2f" % np.mean(LP_weights['reclust_still_bad_match']))

Running on 512 events
Only 512 jets given, will not include LP distortion systematic
Reclustering into 0 subjets?! Something went wrong
Reclustering into 0 subjets?! Something went wrong
Reclustering into 0 subjets?! Something went wrong
Reclustering into 0 subjets?! Something went wrong
Reclustering into 0 subjets?! Something went wrong
Reclustering into 0 subjets?! Something went wrong
Reclustering into 0 subjets?! Something went wrong
Reclustering into 0 subjets?! Something went wrong
Reclustering into 0 subjets?! Something went wrong
Reclustering into 0 subjets?! Something went wrong
Reclustering into 0 subjets?! Something went wrong
Reclustering into 0 subjets?! Something went wrong
Reclustering into 0 subjets?! Something went wrong
Reclustering into 0 subjets?! Something went wrong
Reclustering into 0 subjets?! Something went wrong
Reclustering into 0 subjets?! Something went wrong
Reclustering into 0 subjets?! Something went wrong
Reclustering into 0 subjets?! Something went wro

Reclustering into 0 subjets?! Something went wrong
Reclustering into 0 subjets?! Something went wrong
Reclustering into 0 subjets?! Something went wrong
Reclustering into 0 subjets?! Something went wrong
Reclustering into 0 subjets?! Something went wrong
Reclustering into 0 subjets?! Something went wrong
Reclustering into 0 subjets?! Something went wrong
Reclustering into 0 subjets?! Something went wrong
Reclustering into 0 subjets?! Something went wrong
Reclustering into 0 subjets?! Something went wrong
Reclustering into 0 subjets?! Something went wrong
Reclustering into 0 subjets?! Something went wrong
Reclustering into 0 subjets?! Something went wrong
Reclustering into 0 subjets?! Something went wrong
Reclustering into 0 subjets?! Something went wrong
Reclustering into 0 subjets?! Something went wrong
Reclustering into 0 subjets?! Something went wrong
Reclustering into 0 subjets?! Something went wrong
Reclustering into 0 subjets?! Something went wrong
Reclustering into 0 subjets?! S

In [444]:
LP_weights["nom"].shape

(877,)

In [445]:
LP_weights["nom"]

array([1.0975633 , 0.66041673, 0.81920211, 0.60645347, 0.48256968,
       1.25726798, 0.36125365, 1.282619  , 0.8824103 , 0.44266679,
       0.65468135, 1.70531472, 0.96474908, 0.73904098, 0.77191862,
       0.95687753, 0.55016108, 0.86398988, 0.60618393, 1.02040777,
       0.15509549, 0.80207353, 0.43284322, 1.13367066, 0.86398988,
       3.05649397, 1.5837483 , 3.25095828, 1.00948635, 1.75844057,
       1.72642621, 0.82929625, 0.86398988, 0.82641594, 0.95697574,
       0.74903912, 0.83356191, 1.47564814, 0.32129906, 1.97198001,
       1.00221752, 1.97949806, 0.55129806, 0.82641594, 0.53871669,
       2.89453575, 0.88502117, 0.78576137, 1.82182663, 1.24396422,
       0.92410452, 3.10771249, 0.22323849, 0.89273888, 1.12931753,
       0.57251012, 0.0999611 , 0.85378834, 2.41113616, 1.24017766,
       0.47325022, 0.0999611 , 0.87670452, 0.74973867, 0.76827828,
       1.30817878, 0.8492193 , 0.83678686, 0.55954415, 0.84816345,
       1.00947874, 0.76576038, 0.45028974, 0.91422177, 0.53826

In [236]:
score = score[:max_evts]
score_cut = score > 0.92

###### Use weights to compute efficiency of a cut

#Efficiency of the cut in nominal MC
eff_nom = np.average(score_cut, weights = nom_weights)

#Efficiency of the cut after the Lund Plane reweighting
eff_rw = np.average(score_cut, weights = LP_weights['nom'])

#Nominal 'scale factor'
SF = eff_rw / eff_nom

print("Nominal efficiency %.3f, Corrected efficiency %.3f, SF (corrected / nom) %.3f" % (eff_nom, eff_rw, SF))

#NOTE, because there is kinematic dependence to the correction, it is better to use corrected efficiency computed 
#separately for each MC sample rather than a single 'SF'


######  Compute uncertainties on the efficiency from the various weight variations ##############

#statistical and pt extrapolation uncertainties derived from 100 variations of the weights 
#take std dev to determine unc

nToys = LP_weights['stat_vars'].shape[1]
eff_toys = []
pt_eff_toys = []
for i in range(nToys):
    eff = np.average(score_cut, weights = LP_weights['stat_vars'][:,i])
    eff_toys.append(eff)

    eff1 = np.average(score_cut, weights = LP_weights['pt_vars'][:,i])
    pt_eff_toys.append(eff1)

#Compute stat and pt uncertainty based on variation in the toys
toys_mean = np.mean(eff_toys)
toys_std = np.std(eff_toys)
pt_toys_mean = np.mean(pt_eff_toys)
pt_toys_std = np.std(pt_eff_toys)

#if mean of toys is biased, also include it as an unc (should be zero)
eff_stat_unc = (abs(toys_mean - eff_rw)  + toys_std) 
eff_pt_unc = (abs(pt_toys_mean - eff_rw) + pt_toys_std)

print("Stat variation toys eff. avg %.3f, std dev %.3f" % (toys_mean, toys_std))
print("Pt variation toys eff. avg %.3f, std dev %.3f" % (pt_toys_mean, pt_toys_std))


#Other systematics come from up/down variations of the weights
sys_keys = ['sys', 'bquark', 'prongs', 'unclust', 'distortion']
sys_uncs = dict()

for sys in sys_keys: sys_uncs[sys] = [0.,0.]

#Compute difference in efficiency due to weight variations as uncertainty
def get_uncs(cut, weights_up, weights_down, eff_baseline):
    eff_up =  np.average(cut, weights = weights_up)
    eff_down =  np.average(cut, weights = weights_down)

    unc_up = eff_up - eff_baseline
    unc_down = eff_down - eff_baseline 
    return unc_up, unc_down

for sys in sys_keys:
    unc_up, unc_down = get_uncs(score_cut, LP_weights[sys + '_up'], LP_weights[sys + '_down'], eff_rw)
    sys_uncs[sys] = [unc_up, unc_down]

#Print uncertainty breakdown
eff_str = "Calibrated efficiency  is %.2f +/- %.2f (stat) +/- %.2f (pt)" % (eff_rw, eff_stat_unc, eff_pt_unc )
tot_unc_up = tot_unc_down = eff_stat_unc**2 + eff_pt_unc**2

for sys in sys_keys:
    eff_str += " %.2f/%.2f (%s)" % (sys_uncs[sys][0], sys_uncs[sys][1], sys)
    up_var = max(sys_uncs[sys][0], sys_uncs[sys][1])
    down_var = min(sys_uncs[sys][0], sys_uncs[sys][1])
    tot_unc_up += up_var**2
    tot_unc_down += down_var**2



tot_unc_up = tot_unc_up**0.5
tot_unc_down = tot_unc_down**0.5
print()
#Print final calibrated efficiency and total uncertaintiy
eff_str += "\n Original %.2f, Calibrated %.2f +%.2f/-%.2f \n"  % (eff_nom, eff_rw, tot_unc_up, tot_unc_down)

print(eff_str)
f_ratio.Close()

Nominal efficiency 0.263, Corrected efficiency 0.247, SF (corrected / nom) 0.940
Stat variation toys eff. avg 0.247, std dev 0.014
Pt variation toys eff. avg 0.247, std dev 0.000

Calibrated efficiency  is 0.25 +/- 0.02 (stat) +/- 0.00 (pt) -0.00/0.01 (sys) 0.00/0.00 (bquark) -0.00/-0.00 (prongs) -0.07/0.03 (unclust) nan/nan (distortion)
 Original 0.26, Calibrated 0.25 +nan/-nan 

